# Data aggregation and group by operations

Documentation sources:

* https://www.tutorialspoint.com/python_pandas/
* https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html    
* https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
* https://mode.com/blog/bridge-the-gap-window-functions
    
More advanced topics are discussed in the following sources
* http://benalexkeen.com/resampling-time-series-data-with-pandas/
* https://machinelearningmastery.com/resample-interpolate-time-series-data-python/
* http://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#resampling

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import read_csv
from pandas import concat

## I. Data aggregation

* Data aggregation can be done by:
  * aggregating over (time) windows (`rolling`)
  * aggregating over the columns or rows of the data frame (`aggregate`)
  * computing cumulative sums, products, minimums and maximums (`cumxxx`)  
  

* Details of window functions are described in [Method Chaining Tutorial](./04_pandas_method_chaining.ipynb).
* Method `aggregate` summarises the entrire data frame:
   * `func` specifies aggregation functions which must handle listlike objects 
   * `axis` marks the direction: `0` aggregates over rows, `1` aggregates over columns


* Method `describe` shows standard statistics for each column.
* Method `quantile` allows to compute quantiles for each column.

* Cumulative functions can be computed with:
  * `cumsum`, `cumprod`, `cummin`, `cummax`
  * `expanding.apply(func)` -- for any function that returns a single value

In [2]:
df = DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]})

# Aggregate over rows
display(df.aggregate(sum, axis=0))

# Aggregate over columns
display(df.aggregate(sum, axis=1))

# Use different functions for different columns
# Returns aggregates only for these columns that are present
# It is possible to select more than one function 
display(df.aggregate({'a': 'mean', 'b': np.mean}, axis=0))
display(df.aggregate({'a': ['mean', 'sum', 'median'], 'b': np.mean}, axis=0))
display(df.aggregate(['mean', 'sum', 'median'], axis=0))

# Show standard statistics
display(df.describe())

# Show quantiles for each column 
display(df.quantile(q = [0.1, 0.5, 0.9]))

# Cumulative sum and any other function. 
# Expanding creates expanding windows what must be aggregated with apply 
display(df.cumsum())
display(df.expanding(1).apply(min, raw = False))

a     6
b    15
c    24
dtype: int64

0    12
1    15
2    18
dtype: int64

a    2.0
b    5.0
dtype: float64

,a,b
mean,2.0,5.0
sum,6.0,NaN
median,2.0,NaN


,a,b,c
mean,2.0,5.0,8.0
sum,6.0,15.0,24.0
median,2.0,5.0,8.0


,a,b,c
count,3.0,3.0,3.0
mean,2.0,5.0,8.0
std,1.0,1.0,1.0
min,1.0,4.0,7.0
25%,1.5,4.5,7.5
50%,2.0,5.0,8.0
75%,2.5,5.5,8.5
max,3.0,6.0,9.0


,a,b,c
0.1,1.2,4.2,7.2
0.5,2.0,5.0,8.0
0.9,2.8,5.8,8.8


,a,b,c
0,1,4,7
1,3,9,15
2,6,15,24


,a,b,c
0,1.0,4.0,7.0
1,1.0,4.0,7.0
2,1.0,4.0,7.0


## II. Concatenation

* Sometimes the data is horizontally split among several data frames:
  * For instance, there is a data frame for each year with the same structure.
  
* Methods `df.append` and `pandas.concat` allow to combine such data frames:
  * At this phase, keep indices simple or you might get into trouble.
  * `ignore_index` – allows to merge tables with conflicting indices
  * `verify_integrity` – allows to check against duplicates in the index
  * `sort` – prescribes whether columns are sorted according to their names

* Sometimes you want to sress from which data frame the row comes:
  * `keys` argument allows to specify column values for the index
  * Works only if the original index is preserved.

* Horisontally concatenated data frames may have different columns:
  * Conatenation fails if column names cannot be uniquely combined.

* Vertical concatenation of data frames can be achieved by setting `axis= 1`:
  * Rows are combined based on the index values.
  * If indexes do not match you must specify how indexes are combined:
    * `join='outer'` takes all rows
    * `join='inner'` takes matching rows

In [3]:
df1 = DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]})
df2 = DataFrame({'a': [1, 2, 3], 'b': [1, 2, 3], 'c': [1, 2, 3]})

# Horizontal concatenation
display(concat([df1,df2], ignore_index = True))
display(concat([df1,df2], keys = ['df1','df2']))
display(concat([df1[['b', 'a']], df2[['b', 'c']]], sort = True, ignore_index = True))


# Vertical concatenation
display(concat([df1,df2], axis=1))
display(concat([df1.loc[0:1,:],df2.loc[1:2, :]], axis=1, join = 'outer'))
display(concat([df1.loc[0:1,:],df2.loc[1:2, :]], axis=1, join = 'inner'))

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9
3,1,1,1
4,2,2,2
5,3,3,3


a  b  c
df1 0  1  4  7
    1  2  5  8
    2  3  6  9
df2 0  1  1  1
    1  2  2  2
    2  3  3  3

,a,b,c
0,1.0,4,NaN
1,2.0,5,NaN
2,3.0,6,NaN
3,NaN,1,1.0
4,NaN,2,2.0
5,NaN,3,3.0


,a,b,c,a,b,c
0,1,4,7,1,1,1
1,2,5,8,2,2,2
2,3,6,9,3,3,3


,a,b,c,a,b,c
0,1.0,4.0,7.0,NaN,NaN,NaN
1,2.0,5.0,8.0,2.0,2.0,2.0
2,NaN,NaN,NaN,3.0,3.0,3.0


,a,b,c,a,b,c
1,2,5,8,2,2,2


## III.  Joins

* Different data frames can be joined based on key columns or indices.
* If key columns are pushed to the index then `df.join` is appropriate:
  * Data frames inside the `df.join` call must have key columns as indices.
  * Use `on` argument if key columns are outside of `df.index`.
  * Join type is determined by `how` argument.
  * Arguments `lsuffix` and `rsuffix` are used to disambiguate overlapping columns.
* Otherwise, `df.merge` is more appropriate. It is also more flexible:
  * key columns can be named differently
  * merge type can be validated

In [4]:
df1 = DataFrame({'a': [1, 1, 2, 2], 'b': [1, 2, 1, 2], 'c': [1, 2, 3, 4]}).set_index(['a', 'b']) 
df2 = DataFrame({'a': [1, 1, 2, 2], 'b': [1, 2, 1, 2], 'd': ['a', 'b', 'c', 'd']}).set_index(['a', 'b'])

# Simple join based on the indices
display(df1.join(df2))

# The same join based on merge 
display(df1.reset_index().merge(df2.reset_index(), on = ['a', 'b']))

# You can use different columns in merge  
display(df1.reset_index().merge(df2.reset_index(), left_on = ['a', 'b'], right_on = ['b', 'a']))

# Merge can create many pairs if the join columns have duplicate values  
display(df1.reset_index().merge(df2.reset_index(), on = 'a'))

# Left, right, inner and outer joins are also possible
display(df1.iloc[:2, :].reset_index().merge(df2.iloc[2:, :].reset_index(), on = ['a', 'b'], how = 'left'))
display(df1.iloc[:2, :].reset_index().merge(df2.iloc[2:, :].reset_index(), on = ['a', 'b'], how = 'right'))
display(df1.iloc[:2, :].reset_index().merge(df2.iloc[2:, :].reset_index(), on = ['a', 'b'], how = 'inner'))
display(df1.iloc[:2, :].reset_index().merge(df2.iloc[2:, :].reset_index(), on = ['a', 'b'], how = 'outer'))

c  d
a b      
1 1  1  a
  2  2  b
2 1  3  c
  2  4  d

,a,b,c,d
0,1,1,1,a
1,1,2,2,b
2,2,1,3,c
3,2,2,4,d


,a_x,b_x,c,a_y,b_y,d
0,1,1,1,1,1,a
1,1,2,2,2,1,c
2,2,1,3,1,2,b
3,2,2,4,2,2,d


,a,b_x,c,b_y,d
0,1,1,1,1,a
1,1,1,1,2,b
2,1,2,2,1,a
3,1,2,2,2,b
4,2,1,3,1,c
5,2,1,3,2,d
6,2,2,4,1,c
7,2,2,4,2,d


,a,b,c,d
0,1,1,1,NaN
1,1,2,2,NaN


,a,b,c,d
0,2,1,NaN,c
1,2,2,NaN,d


,a,b,c,d


,a,b,c,d
0,1,1,1.0,NaN
1,1,2,2.0,NaN
2,2,1,NaN,c
3,2,2,NaN,d


## IV. Group by operations

* The chain of group by methods consists of three phases:
  * splitting the data
  * aggregation over the groups
  * combining the results into the outcome
* The chain of group by methods allows to accomplish three types of tasks:
  * aggregation tasks – compute summary statistics for each group
  * data transformation tasks – apply adjustment methods for each group
  * filtration tasks – filter out some groups based on their contents  

* Pandas divides group by operations into following operations
  * data splitting
  * aggregation and iteration over groups
  * filtering and selection

Further details can be found:
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

### Data  splitting

* Data splitting is implemented by `df.group_by`:
  * `by` is either a list of columns or a function that computes group identifiers
  * `level` specifies how many or what levels of a multi-index are used for grouping
  * `axis` allows to split the data frame over columns which is not a good style 
  
* The resulting object provide means for iteration and aggregation.
* **Important**: All rows in which group key contains `NA` and `NaT` values are excluded:
  * Safe grouping is possible only over complete columns.

In [5]:
df = read_csv('billboard.csv')

# Simple grouping based on single column
grouped = df.groupby('performer')
display(grouped.aggregate({'week': lambda df: df.shape[0]})
        .rename(columns={'week': 'number_of_weeks'})
        .sort_values('number_of_weeks', ascending = False)
        .head(5))

# Same grouping using multi-index
grouped = df.set_index(['performer', 'song']).groupby(level = 'performer')
display(grouped.aggregate({'week': lambda df: df.shape[0]})
        .rename(columns={'week': 'number_of_weeks'})
        .sort_values('number_of_weeks', ascending = False)
        .head(5))

# Grouping using two columns
grouped = df.set_index(['performer', 'song']).groupby(level = [0, 1])
display(grouped.aggregate({'week': lambda df: df.shape[0]})
        .rename(columns={'week': 'number_of_weeks'})
        .sort_values('number_of_weeks', ascending = False)
        .head(5))

,number_of_weeks
performer,
Drake,168
XXXTENTACION,110
Post Malone,97
Ariana Grande,80
Imagine Dragons,73


,number_of_weeks
performer,
Drake,168
XXXTENTACION,110
Post Malone,97
Ariana Grande,80
Imagine Dragons,73


,,number_of_weeks
performer,song,
Khalid & Normani,Love Lies,44
Bebe Rexha & Florida Georgia Line,Meant To Be,43
Lauv,I Like Me Better,41
"Zedd, Maren Morris & Grey",The Middle,40
Ed Sheeran,Perfect,40


#### Advanced example: group cells are too small

* Each column is a group. We want to sum over the entire column.
* Grouping with `axis=1` allows us to iterate over columns but we can access a single row at a time.
* Staking is needed to widen the group and to reduce the number of rows.
* This design pattern is universal – reshape indices before grouping if you need to sum over several rows or columns.

In [6]:
df = read_csv('estonian_population_1.csv').set_index(['Gender', 'Age'])
display(df.iloc[0:5,0:5])

# Grouping over rows as the data is in wrong format
# Unfortunately each aggregation cell is too small -- contains a single value
display(df.reset_index(level = 'Age', drop = True).groupby(level=0, axis=1).count().iloc[0:5,0:5])

# To resolve this issue we must push Age to a column index
# Now each aggregation cell has all age values and only three rows are left
display(df.unstack('Age').sort_index().iloc[0:5,0:5])
grouped = df.unstack('Age').groupby(level=0, axis=1)
display(grouped.aggregate(sum).iloc[:, 0:10])

,,1950,1951,1952,1953,1954
Gender,Age,,,,,
All,0,19310,18841,19356,19563,19239
Female,0,9303,9042,9298,9458,9260
Male,0,10007,9799,10058,10105,9979
All,1,18162,18251,17743,18368,19069
Female,1,8856,8889,8609,8910,9291


,1950,1951,1952,1953,1954
Gender,,,,,
All,1,1,1,1,1
Female,1,1,1,1,1
Male,1,1,1,1,1
All,1,1,1,1,1
Female,1,1,1,1,1


1950                            
Age         0      1      2      3      4
Gender                                   
All     19310  18162  16928  16540  13620
Female   9303   8856   8178   8117   6680
Male    10007   9306   8750   8423   6940

,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959
Gender,,,,,,,,,,
All,1017727,1044555,1068055,1087493,1114991,1132453,1145344,1159253,1172770,1185142
Female,586548,598711,609216,617871,630170,637649,643083,652950,658642,663546
Male,431179,445844,458839,469622,484821,494804,502261,506303,514128,521596


### Selection and iteration

* The data corresponding to a group can be viewed by `grouped.get_group(group_key)`.
* If you do not know group key values you can use `grouped.get_group(list(grouped.groups)[0]`.
* Iteration over groups can be done by iteration over `grouped`.

In [7]:
# Another way to see what inputs go into aggregation and filtration
for name, group in grouped:
    print(name)
    display(group)
    break

1950


1950                                                                 \
Age        0      1      2      3      4      5      6      7      8      9    
Gender                                                                         
All     19310  18162  16928  16540  13620  13204  12928  14975  18242  16259   
Female   9303   8856   8178   8117   6680   6552   6280   7361   8891   7911   
Male    10007   9306   8750   8423   6940   6652   6648   7614   9351   8348   

        ...                                                              
Age     ...    75    76    77    78    79    80    81    82    83    84  
Gender  ...                                                              
All     ...  5262  4690  4075  3416  3383  2539  2097  1832  1736  1455  
Female  ...  3551  3181  2786  2381  2295  1814  1453  1263  1233  1023  
Male    ...  1711  1509  1289  1035  1088   725   644   569   503   432  

[3 rows x 85 columns]

### Aggregation

* The `grouped` object returned by `df.groupby` implements standard aggregation functions:
  * statistical functions `count`, `min`, `max`, `sum`, `describe`, ...
  * visualisation functions `boxplot`, `hist`, `plot`
  * cumulative functions `cumsum`, `cummax`, ...
  * information about each group `ngroup`, `cumcount`
  * summary information about groups `ngroups`, `groups`

* Custom aggregation functions can be defined using `aggregate`:
  * use list of functions to apply functions to all columns 
  * use dictionary to apply different functions to different columns
  * functions must be able to process series-like objects
  * use ordered dictionary to fix the order of columns
  * Resulting column names will be named after functions if there are more than one function.
  * As a result, column index may become multi-index with original column name on top.

In [8]:
df = read_csv('billboard.csv')
grouped = df.groupby('performer')

# Apply same functions to all columns
display(grouped.aggregate([min, max, len]).rename(columns={'len': 'count'}).head())

# Apply different functions to different columns
display(grouped.aggregate({'position': [min, max], 'weeks_on_chart':max})
        .sort_values([('position', 'min'), ('weeks_on_chart', 'max')], ascending = [True, False])
        .head())

# Use custom function to show all songs
display(grouped.aggregate({'song': [lambda x: set(x), 'nunique']})
        .sort_values(('song','nunique'), ascending = False)
        .head(10)) 

# Count the number of songs based on the first letter of the performer name
# Gruping is based on function that extracts the first letter from the index 
grouped = df.set_index('performer').groupby(lambda x: x.upper()[0])
display(grouped.aggregate({'song': [lambda x: set(x), 'nunique']})
        .reset_index()
        .rename(columns={'index': 'First letter'})
        .sort_values(('song','nunique'), ascending = False)
        .head(10))

week                   position      \
                                       min         max count      min max   
performer                                                                   
2 Chainz Featuring YG & Offset  2018-04-14  2018-04-21     2       96  99   
2 Chainz, Drake & Quavo         2018-06-30  2018-07-07     2       53  60   
21 Savage                       2018-01-06  2018-02-03     5       28  47   
5 Seconds Of Summer             2018-03-10  2018-12-29    30        7  96   
6LACK Featuring J. Cole         2018-09-29  2018-09-29     1       76  76   

                                                     song  \
                               count                  min   
performer                                                   
2 Chainz Featuring YG & Offset     2                Proud   
2 Chainz, Drake & Quavo            2         Bigger > You   
21 Savage                          5         Bank Account   
5 Seconds Of Summer               30        Want You Back   
6LACK Featuring J. Cole            1  Pretty Little Fears   

                                                          instance  ...        \
                                                max count      min  ... count   
performer                                                           ...         
2 Chainz Featuring YG & Offset                Proud     2        1  ...     2   
2 Chainz, Drake & Quavo                Bigger > You     2        1  ...     2   
21 Savage                              Bank Account     5        1  ...     5   
5 Seconds Of Summer                      Youngblood    30        1  ...    30   
6LACK Featuring J. Cole         Pretty Little Fears     1        1  ...     1   

                               previous_position           peak_position      \
                                             min max count           min max   
performer                                                                      
2 Chainz Featuring YG & Offset                 -  96     2            96  96   
2 Chainz, Drake & Quavo                        -  53     2            53  53   
21 Savage                                     28  38     5            12  12   
5 Seconds Of Summer                            -  96    30             7  96   
6LACK Featuring J. Cole                        -   -     1            76  76   

                                     weeks_on_chart            
                               count            min max count  
performer                                                      
2 Chainz Featuring YG & Offset     2              1   2     2  
2 Chainz, Drake & Quavo            2              1   2     2  
21 Savage                          5             24  28     5  
5 Seconds Of Summer               30              1  29    30  
6LACK Featuring J. Cole            1              1   1     1  

[5 rows x 21 columns]

position     weeks_on_chart
                                         min max            max
performer                                                      
Ed Sheeran                                 1  29             58
Camila Cabello Featuring Young Thug        1  39             45
Post Malone Featuring Ty Dolla $ign        1  48             39
XXXTENTACION                               1  99             38
Cardi B, Bad Bunny & J Balvin              1  42             37

song        
                                                      <lambda_0> nunique
performer                                                               
Drake          {March 14, Mob Ties, Elevate, 8 Out Of 10, San...      23
Travis Scott   {No Bystanders, Butterfly Effect, Sicko Mode, ...      17
XXXTENTACION   {Difference (Interlude), Numb, Changes, Everyb...      17
Post Malone    {Blame It On Me, Sugar Wraith, Candy Paint, 92...      14
Lil Wayne      {Dedicate, Uproar, Demon, Open Safe, Perfect S...      13
J. Cole        {Motiv8, Album Of The Year (Freestyle), Kevin'...      11
Ariana Grande  {God Is A Woman, Everytime, Imagine, No Tears ...      10
Lil Baby       {Deep End, Pure Cocaine, Crush A Lot, Word On ...       8
Eminem         {Kamikaze, Fall, Greatest, Normal, Venom, Step...       8
Cardi B        {Thru Your Phone, Be Careful, Money, Money Bag...       7

performer                                               song        
                                                     <lambda_0> nunique
14         L  {Falling Down, Sunrise, Sunburn, Sunset, NBAYo...      94
22         T  {Jackie Chan, Boss, Nice, Yosemite, Hurt You, ...      63
15         M  {Dreamcatcher, Up To Something, No More, Kiss ...      62
6          D  {After Dark, March 14, Mob Ties, Elevate, Kiss...      53
13         K  {Let It Sing, Feel The Love, ZEZE, OTW, Wouldn...      51
5          C  {Thru Your Phone, One Kiss, Be Careful, She Ba...      44
12         J  {Album Of The Year (Freestyle), Lean Wit Me, S...      42
18         P  {Blame It On Me, Backin' It Up, 92 Explorer, I...      38
4          B  {Jingle Bell Rock, When The Party's Over, That...      35
7          E  {Kamikaze, Shape Of You, Trip, Normal, Venom, ...      23

## Data transformations

* Data transformations do not change the number of observations.
* Instead, you want to adjust values in the data frame corresponding to each group.
* There are two mechanisms for that: `grouped.apply` and `grouped.transform`.
* The method `grouped.apply(func)` provides access to the entire data frame:
  * You must make sure that the function returns a data frame.
  * Make sure that group key columns are preserved.
* The method `grouped.transform(func)` provides access to individual columns:
  * You must make sure that it returns a single value or series of same size.
  * Make sure that group key columns are preserved.
* As `grouped.transform` applies the same transformation for all columns:
  * non-numerical columns must describe the group
  * numerical columns require some correctness

In [9]:
# Let's compute how much the position has changed compared to the average in the group

# Boring data conversion
grouped = (df.assign(week = pd.to_datetime(df['week']),
                    position = pd.to_numeric(df['position']),
                    previous_position = pd.to_numeric(df['previous_position'], errors = 'coerce'))
           .groupby('week'))

In [10]:
grouped.apply(lambda df: df.assign(position_change = lambda df: df['position'] - df['previous_position'],
                                  mean_change = lambda df: df['position_change'].mean(skipna =True), 
                                           adjusted_change = lambda df: df['position_change'] - df['mean_change']))

,week,position,performer,song,instance,previous_position,peak_position,weeks_on_chart,position_change,mean_change,adjusted_change
0,2018-01-06,1,Ed Sheeran,Perfect,2,1.0,1,17,0.0,0.590909,-0.590909
1,2018-01-06,2,Post Malone Featuring 21 Savage,Rockstar,1,2.0,1,14,0.0,0.590909,-0.590909
2,2018-01-06,3,Camila Cabello Featuring Young Thug,Havana,2,3.0,2,19,0.0,0.590909,-0.590909
3,2018-01-06,4,G-Eazy Featuring A$AP Rocky & Cardi B,No Limit,1,5.0,4,15,-1.0,0.590909,-1.590909
4,2018-01-06,5,Imagine Dragons,Thunder,1,6.0,4,34,-1.0,0.590909,-1.590909
...,...,...,...,...,...,...,...,...,...,...,...
5195,2018-12-29,96,A Boogie Wit da Hoodie,Look Back At It,1,95.0,95,2,1.0,0.313253,0.686747
5196,2018-12-29,97,Meek Mill Featuring Ella Mai,24/7,1,86.0,54,3,11.0,0.313253,10.686747
5197,2018-12-29,98,Meek Mill Featuring Fabolous & Anuel AA,Uptown Vibes,1,79.0,39,3,19.0,0.313253,18.686747
5198,2018-12-29,99,Ski Mask The Slump God Featuring Juice WRLD,Nuketown,1,98.0,63,3,1.0,0.313253,0.686747


In [11]:
# Data transformation with apply and some cleanup
display(grouped
    .apply(
        lambda df: df
        .assign(
            position_change = lambda df: df['position'] - df['previous_position'], 
            mean_change = lambda df: df['position_change'].mean(skipna =True), 
            adjusted_change = lambda df: df['position_change'] - df['mean_change'])
        .pipe(lambda df: df[['week','performer', 'song', 'position_change', 'adjusted_change']])
    )
    .sort_values(['week','adjusted_change'], ascending = [True, False])
    .head(10)
)        

,week,performer,song,position_change,adjusted_change
42,2018-01-06,Eminem Featuring Ed Sheeran,River,32.0,31.409091
66,2018-01-06,6ix9ine,Kooda,13.0,12.409091
83,2018-01-06,Lil Baby,My Dawg,13.0,12.409091
84,2018-01-06,Famous Dex Featuring A$AP Rocky,Pick It Up,11.0,10.409091
98,2018-01-06,Zayn Featuring Sia,Dusk Till Dawn,11.0,10.409091
23,2018-01-06,G-Eazy & Halsey,Him & I,9.0,8.409091
34,2018-01-06,Lil Uzi Vert Featuring Nicki Minaj,The Way Life Goes,8.0,7.409091
45,2018-01-06,Burl Ives,A Holly Jolly Christmas,8.0,7.409091
52,2018-01-06,Offset & Metro Boomin,Ric Flair Drip,8.0,7.409091
31,2018-01-06,J Balvin & Willy William Featuring Beyonce,Mi Gente,7.0,6.409091


In [12]:
# Let's correct previous position against median position

# Boring data conversion
grouped = (df.assign(week = pd.to_datetime(df['week']),
                    position = pd.to_numeric(df['position']),
                    previous_position = pd.to_numeric(df['previous_position'], errors = 'coerce'))
           .pipe(lambda df: df[['week', 'position', 'previous_position']])
           .set_index('week')
           .groupby(level='week'))

# Data transformation. Note that we will lose performer and song here
display(grouped.transform(lambda x: x - x.median(skipna =True)).head(10))

,position,previous_position
week,,
2018-01-06,-49.5,-44.5
2018-01-06,-48.5,-43.5
2018-01-06,-47.5,-42.5
2018-01-06,-46.5,-40.5
2018-01-06,-45.5,-39.5
2018-01-06,-44.5,-41.5
2018-01-06,-43.5,-38.5
2018-01-06,-42.5,-37.5
2018-01-06,-41.5,-35.5


### Subgroup filtering

* Method `grouped.filter(func)` allows to filter out data about individual groups.
* Filter `func` takes in the data frame corresponding to each group and outputs a Boolean decision.

In [13]:
# Let's keep only data about these performers who are in the chart each week
display(df.groupby('performer').filter(lambda df: df[['week']].nunique() == 52)
        .sort_values(['performer', 'week'])
        .head(10))

# Let's keep data about songs that have been at least 12 weeks in the top 3
display(df.groupby(['performer', 'song']).filter(lambda df: sum(df['position'] <= 3) >= 12)
        .sort_values(['week', 'position'])
        .head(10))

,week,position,performer,song,instance,previous_position,peak_position,weeks_on_chart
4,2018-01-06,5,Imagine Dragons,Thunder,1,6,4,34
24,2018-01-06,25,Imagine Dragons,Believer,1,40,4,46
103,2018-01-13,4,Imagine Dragons,Thunder,1,5,4,35
115,2018-01-13,16,Imagine Dragons,Believer,1,25,4,47
205,2018-01-20,6,Imagine Dragons,Thunder,1,4,4,36
229,2018-01-20,30,Imagine Dragons,Believer,1,16,4,48
305,2018-01-27,6,Imagine Dragons,Thunder,1,6,4,37
325,2018-01-27,26,Imagine Dragons,Believer,1,30,4,49
408,2018-02-03,9,Imagine Dragons,Thunder,1,6,4,38
427,2018-02-03,28,Imagine Dragons,Believer,1,26,4,50


,week,position,performer,song,instance,previous_position,peak_position,weeks_on_chart
0,2018-01-06,1,Ed Sheeran,Perfect,2,1,1,17
100,2018-01-13,1,Ed Sheeran,Perfect,2,1,1,18
200,2018-01-20,1,Ed Sheeran,Perfect,2,1,1,19
301,2018-01-27,2,Ed Sheeran,Perfect,2,1,1,20
400,2018-02-03,1,Drake,God's Plan,1,-,1,1
401,2018-02-03,2,Ed Sheeran,Perfect,2,2,1,21
500,2018-02-10,1,Drake,God's Plan,1,1,1,2
501,2018-02-10,2,Ed Sheeran,Perfect,2,2,1,22
600,2018-02-17,1,Drake,God's Plan,1,1,1,3
601,2018-02-17,2,Ed Sheeran,Perfect,2,2,1,23


## V. Reordering operations

* The order of a data frame rows can be changed by reindexing or value-based sorting
* Reindexing is done by explicitly specifing the order of index elements:
  * `reindex` – requires the explicit order
  * `reindex_like` – gets order from other data frame or series
* Data frame can be sorted according to index or according to columns
  * `sort_index` – sorts based on selected columns in the index
  * `sort_values` – sorts based on selected columns in the data frame
* Both functions allow to determine the ordering for each column separately.

In [14]:
# Show most recent songs in reverse alphabetical order
display(df.head(10).sort_values(['weeks_on_chart', 'performer'], ascending = [True, False]))

# Same task with index columns
display(df.head(10).set_index(['weeks_on_chart', 'performer']).sort_index(ascending = [True, False]))

,week,position,performer,song,instance,previous_position,peak_position,weeks_on_chart
6,2018-01-06,7,"Migos, Nicki Minaj & Cardi B",MotorSport,1,7,6,8
1,2018-01-06,2,Post Malone Featuring 21 Savage,Rockstar,1,2,1,14
7,2018-01-06,8,Sam Smith,Too Good At Goodbyes,1,8,4,15
5,2018-01-06,6,Lil Pump,Gucci Gang,1,4,3,15
3,2018-01-06,4,G-Eazy Featuring A$AP Rocky & Cardi B,No Limit,1,5,4,15
8,2018-01-06,9,Halsey,Bad At Love,1,10,8,16
0,2018-01-06,1,Ed Sheeran,Perfect,2,1,1,17
2,2018-01-06,3,Camila Cabello Featuring Young Thug,Havana,2,3,2,19
9,2018-01-06,10,Cardi B,Bodak Yellow (Money Moves),1,12,1,25
4,2018-01-06,5,Imagine Dragons,Thunder,1,6,4,34


week  position  \
weeks_on_chart performer                                                     
8              Migos, Nicki Minaj & Cardi B           2018-01-06         7   
14             Post Malone Featuring 21 Savage        2018-01-06         2   
15             Sam Smith                              2018-01-06         8   
               Lil Pump                               2018-01-06         6   
               G-Eazy Featuring A$AP Rocky & Cardi B  2018-01-06         4   
16             Halsey                                 2018-01-06         9   
17             Ed Sheeran                             2018-01-06         1   
19             Camila Cabello Featuring Young Thug    2018-01-06         3   
25             Cardi B                                2018-01-06        10   
34             Imagine Dragons                        2018-01-06         5   

                                                                            song  \
weeks_on_chart performer                                                           
8              Migos, Nicki Minaj & Cardi B                           MotorSport   
14             Post Malone Featuring 21 Savage                          Rockstar   
15             Sam Smith                                    Too Good At Goodbyes   
               Lil Pump                                               Gucci Gang   
               G-Eazy Featuring A$AP Rocky & Cardi B                    No Limit   
16             Halsey                                                Bad At Love   
17             Ed Sheeran                                                Perfect   
19             Camila Cabello Featuring Young Thug                        Havana   
25             Cardi B                                Bodak Yellow (Money Moves)   
34             Imagine Dragons                                           Thunder   

                                                      instance  \
weeks_on_chart performer                                         
8              Migos, Nicki Minaj & Cardi B                  1   
14             Post Malone Featuring 21 Savage               1   
15             Sam Smith                                     1   
               Lil Pump                                      1   
               G-Eazy Featuring A$AP Rocky & Cardi B         1   
16             Halsey                                        1   
17             Ed Sheeran                                    2   
19             Camila Cabello Featuring Young Thug           2   
25             Cardi B                                       1   
34             Imagine Dragons                               1   

                                                     previous_position  \
weeks_on_chart performer                                                 
8              Migos, Nicki Minaj & Cardi B                          7   
14             Post Malone Featuring 21 Savage                       2   
15             Sam Smith                                             8   
               Lil Pump                                              4   
               G-Eazy Featuring A$AP Rocky & Cardi B                 5   
16             Halsey                                               10   
17             Ed Sheeran                                            1   
19             Camila Cabello Featuring Young Thug                   3   
25             Cardi B                                              12   
34             Imagine Dragons                                       6   

                                                      peak_position  
weeks_on_chart performer                                             
8              Migos, Nicki Minaj & Cardi B                       6  
14             Post Malone Featuring 21 Savage                    1  
15             Sam Smith                                          4  
               Lil Pump                                           3  
            